In [1]:
import pandas as pd, numpy as np, re
from rapidfuzz import process, fuzz,utils
np.set_printoptions(threshold=np.inf)

pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\Kroger\\Data\\' 
path_atb = path + 'ATB\\'
path_atb_yue = path + 'ATB\\atb_yue\\'

In [2]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
pim_flavors = pd.read_csv(path_atb + 'PIM_DATA_WITH_FLAVOR_ATTB_100323.csv')
flavors_COM = pd.DataFrame(pim_flavors.groupby('COM_CD').apply(lambda x: '|'.join(x.Flavor_Code.unique())), columns= ['UNIQUE FLAVORS']).reset_index()

In [34]:
#pim_df = pd.read_csv(path + 'PIM_Data_New_50_82Mn.zip')
pim_load = pd.read_csv(path_atb + 'PIM_DATA_WITH_FLAVOR_ATTB_100323.csv')
pim_load = pim_load.merge(pim_df, how = 'right', on = list(pim_df.columns))
#gtin_itm_id = pd.read_csv(path + 'ITM_NO_GTIN_100323.csv')
#gtin_itm_id = pd.read_csv(path + 'ITM_ID_GTIN_Map_100323.csv')
other_flavor = pd.read_csv(path_atb + 'OtherFlavors.csv')
other_flavor['GTIN_NO'] = other_flavor.ITM_ID.map(dict(zip(gtin_itm_id.ITM_ID, gtin_itm_id.GTIN_NO)))
pim_load = pim_load.merge(other_flavor, how = 'left', on = 'GTIN_NO')

In [35]:
flavored_COM_CDs = flavors_COM[~flavors_COM['UNIQUE FLAVORS'].isin(['NONE','OTHER','MISCELLANEOUS'])].COM_CD.unique()
len(flavored_COM_CDs)

372

In [36]:
pim_load[(pim_load.COM_CD.isin(flavors_COM.COM_CD.unique())) & (~pim_load.Flavor_Code.isin(['NONE']))].shape

(1796229, 19)

In [37]:
pim_load['Flavor_Code'].value_counts().sum()

20921

In [38]:
pim_load['Flavor_Code'] = pim_load.Flavor_Code.fillna('NONE')
pim_load[~pim_load.Flavor_Code.isin(['NONE','OTHER','MISECLLANEOUS'])].shape

(14415, 19)

In [39]:
pim_load  = pim_load[pim_load.COM_CD.isin(flavored_COM_CDs)]
ic_df = pd.read_csv(path_atb + 'ICECREAM_FLAVORS_101223_pipe_delim.csv', sep = '|')
ice_cream_GTINs = ic_df.GTIN_NO.unique()
pim_load = pim_load[~pim_load.GTIN_NO.isin(ice_cream_GTINs)]

In [40]:
pim_load.shape

(1465017, 19)

In [41]:
pim_flavors = pim_flavors[pim_flavors.GTIN_NO.isin(pim_load.GTIN_NO.unique())]
flavors_DPT = pd.DataFrame(pim_flavors.groupby('DPT_CD').apply(lambda x: '|'.join(x.Flavor_Code.unique())), columns= ['UNIQUE FLAVORS']).reset_index()
flavored_DPT_CDs = flavors_DPT[~flavors_DPT['UNIQUE FLAVORS'].isin(['NONE','OTHER','MISCELLANEOUS'])].DPT_CD.unique()
len(flavored_DPT_CDs)

37

In [42]:
replacements = {          
 ',': ' ',
 '/': ' ',
 '>':' ',
'<': ' '}


In [43]:
### Remove substrings like 'CHOCOLATE' if 'CHOCOLATE CHIP' is also present as a top result
def remove_substrings(string_list):
    str_df = pd.DataFrame(string_list, columns= ['Sentence'])
    str_df['Word Length'] = str_df.Sentence.apply(lambda x: len(x.split(' ')))
    str_df['Drop'] = ''
    str_df.sort_values('Word Length', inplace= True)
    str_df.reset_index(drop = True, inplace= True)
    for j in range(len(str_df)):
        str_1 = str_df.Sentence.loc[j]
        for i in range(len(str_df)-(j +1)):
            str_2 = str_df.Sentence.loc[i+j+1]
            if str_1 in str_2: 
                str_df.Drop.loc[j] = 'Yes'
                break
        if len(str_df) <2: 
            break
    return str_df[str_df.Drop != 'Yes'].Sentence.tolist()

In [44]:
#MAtch using 'token set ratio' with threshold
def flavor_match(item, flavor_list_org, cutoff = 90):
    #devowelized = [devowelize(i) for i in flavor_list_org]
    #derepeated = [remove_adjacent_repititions(i) for i in flavor_list_org]
    flavor_list = flavor_list_org #+ devowelized
    #flavor_list += derepeated
    #abbrev_maps = dict(zip(devowelized+ derepeated, flavor_list_org + flavor_list_org))
    flavor_no_spaces = [ i for i in flavor_list_org if not '' in i]
    flavor_with_spaces = [ i for i in flavor_list_org if i not in flavor_no_spaces]
    abbrev_maps = {}
    for i in flavor_no_spaces:
        for j in flavor_with_spaces:
            if j.replace(' ','') == i:
                abbrev_maps[i] = j
    matched = process.extract(item, flavor_list, score_cutoff= cutoff, scorer = fuzz.token_set_ratio, processor=utils.default_process, limit= 7)
    flavor_shortlist = remove_substrings([i[0] for i in matched])
    matched = [i for i in matched if i[0] in flavor_shortlist]
    if len( flavor_shortlist) >0:
        max_score = matched[0][1]
        final_match = [ i for i in matched if i[1]== max_score]
        match_list = [i[0] for i in final_match]
        match_list = [abbrev_maps[i] if i in abbrev_maps.keys() else i for i in match_list]
        rematched = process.extract(item, match_list,  scorer = fuzz.partial_ratio, processor=utils.default_process, limit = 3)
        re_max_score = rematched[0][1]
        final_rematch = [ i for i in rematched if i[1]== re_max_score]
        match_list =  [i[0] for i in final_rematch]
    
        '''
        match_list = [flavor_maps[i] if i in flavor_maps.keys() else i for i in match_list]
        if set(match_list) == {'STRAWBERRY','FRUIT'}: 
            match_list = ['FRUIT']#Drop Strawberry if fruit is the only other flavor
        if  set(match_list) != {'FRUIT'} : match_list = [ i for i in match_list if i != 'FRUIT']#Drop Fruit if it appears with something else
 
        if matched in multiflavor_dict.keys(): #Mapping multi-flavors to intended labels
            matched = multiflavor_dict[matched]
        '''
        matched = ','.join(sorted(list(set(pd.Series(match_list)))))   
    else: matched = 'OTHER'
    return matched

### Match within COM

In [45]:
###
"""
flavors_df = pd.DataFrame()
for com in flavored_COM_CDs[:5]:
    pim_com = pim_load[pim_load.COM_CD == com]
    flavor_list = [i.replace('_',' ') for i in pim_com['Flavor_Code'].unique() if not i in  ['OTHER','NONE','MISCELLANEOUS']]
    pim_com['Detected_Flavor(s)']  =  \
                   pim_com.VND_ECOM_DSC.str.upper().replace(replacements, regex = True).apply(lambda x:flavor_match(x, flavor_list_org= flavor_list + [i.replace(" ","") for  i in  flavor_list] ))   
    flavors_df = flavors_df.append(pim_com)
"""    

'\nflavors_df = pd.DataFrame()\nfor com in flavored_COM_CDs[:5]:\n    pim_com = pim_load[pim_load.COM_CD == com]\n    flavor_list = [i.replace(\'_\',\' \') for i in pim_com[\'Flavor_Code\'].unique() if not i in  [\'OTHER\',\'NONE\',\'MISCELLANEOUS\']]\n    pim_com[\'Detected_Flavor(s)\']  =                     pim_com.VND_ECOM_DSC.str.upper().replace(replacements, regex = True).apply(lambda x:flavor_match(x, flavor_list_org= flavor_list + [i.replace(" ","") for  i in  flavor_list] ))   \n    flavors_df = flavors_df.append(pim_com)\n'

In [46]:
#flavors_df.Flavor_Code.unique()

In [47]:
#flavors_df['Detected_Flavor(s)'].value_counts()

In [48]:
#flavors_df.shape

In [49]:
""" flavors_df[['GTIN_NO','COM_DSC','COM_CD','SUBCOM_DSC','SUBCOM_CD','VND_ECOM_DSC','Flavor_Code','Detected_Flavor(s)']]\
.to_csv('FLAVORS_NON_ICE_CREAM_102123_sample.csv', index = None)
"""

" flavors_df[['GTIN_NO','COM_DSC','COM_CD','SUBCOM_DSC','SUBCOM_CD','VND_ECOM_DSC','Flavor_Code','Detected_Flavor(s)']].to_csv('FLAVORS_NON_ICE_CREAM_102123_sample.csv', index = None)\n"

### Match within DPT

In [50]:
#flavors_DPT_df['Detected_Flavor(s)'].value_counts()

In [51]:
#100*flavors_DPT_df['Detected_Flavor(s)'].value_counts()/len(flavors_DPT_df)

In [52]:
#100*flavors_DPT_df['Flavor_Code'].value_counts()/len(flavors_DPT_df)

In [53]:
flavors_DPT = pd.DataFrame(pim_flavors.groupby('DPT_DSC').apply(lambda x: ','.join(x.Flavor_Code.unique())), columns= ['UNIQUE FLAVORS']).reset_index()
flavored_DPT_DSCs = flavors_DPT[~flavors_DPT['UNIQUE FLAVORS'].isin(['NONE','OTHER','MISCELLANEOUS'])].DPT_DSC.unique()
len(flavored_DPT_DSCs)

37

In [54]:
flavors_DPT = flavors_DPT[flavors_DPT.DPT_DSC.isin(flavored_DPT_DSCs)].reset_index(drop = True)
flavors_DPT['UNIQUE FLAVOR COUNT'] = flavors_DPT['UNIQUE FLAVORS'].\
apply(lambda x: len(','.join([i for i in x.split(',') if not i in ['NONE','OTHER','MISCELLANEOUS']]).split(',')))

In [55]:
flavors_DPT['ITEM_WITH_FLAVOR_COUNT'] = flavors_DPT.DPT_DSC.map(dict(pim_flavors.groupby('DPT_DSC').size()))

In [56]:
flavors_DPT['TOTAL_ITEM_COUNT'] = flavors_DPT.DPT_DSC.map(dict(pim_load.drop_duplicates('GTIN_NO').groupby('DPT_DSC').size()))

In [57]:
flavors_DPT.to_excel('Flavors_DEPT_102523.xlsx')

In [58]:
DPT_remove = ['CARDS','GM-MISC','HOUSEWARES','MEAT/SEAF WRAP','NF NON-EDIBLE']
DPT_review = ['RX-OTC','SPECIALTY MEAT','WINE']#Isolate lower levels for flavor relevant items

In [59]:
flavors_DPT = flavors_DPT[~flavors_DPT.DPT_DSC.isin(DPT_remove + DPT_review)]

In [60]:
flavors_DPT.TOTAL_ITEM_COUNT.sum()

1067312

In [61]:
flavors_DPT.ITEM_WITH_FLAVOR_COUNT.sum()

20473

In [63]:
#Match within DPT
#flavors_DPT_df = pd.DataFrame()
#for dpt in ['COFFEE SHOP']:#flavors_DPT.DPT_DSC.unique():
for dpt in ['BAKERY']:#flavors_DPT.DPT_DSC.unique():    
    print(dpt)
    pim_dpt = pim_load[pim_load.DPT_DSC == dpt]
    flavor_list = [i.replace('_',' ') for i in pim_load['Flavor_Code'].unique() if not i in  ['OTHER','NONE','MISCELLANEOUS']]
    pim_dpt['Detected_Flavor(s)']  =  \
                   pim_dpt.VND_ECOM_DSC.str.upper().replace(replacements, regex = True).apply(lambda x:flavor_match(x, flavor_list_org= flavor_list + [i.replace(" ","") for  i in  flavor_list]))   
    pim_dpt.to_excel(path_atb + 'FLAVOR_MAPPING_'+dpt+'_DEPARTMENT_syorg.xlsx', sheet_name= dpt)

BAKERY


In [ ]:
flavors_DPT.DPT_DSC.unique()